In [13]:
from modules.engine import SolverPool, SolverSelectionStrategy, MySolver, Query
from modules.log_reader import read_qsym_log
import logging

In [14]:
logging.basicConfig(level=logging.INFO)

In [15]:
class NegationSolverSelectionStrategy(SolverSelectionStrategy):
    def get_solver(self, found_solver: MySolver | None, query: Query) -> MySolver:
        if found_solver is None:
            logging.info("No solver available, Creating a new one")
            solver = self.create_empty_solver()
            solver.upgrade(query.path[:-1], query.constraints[:-1])
            solver.upgrade(query.path, query.constraints)
            return solver

        solver = found_solver
        if found_solver.stack_path_len < len(query.path) - 1:
            logging.info("Creating a new solver for a longer path")
            solver = self.create_empty_solver()
            solver.copy_from(found_solver)
            solver.upgrade(query.path[solver.stack_path_len:-1],
                           query.constraints[solver.constraint_count:-1])
        return solver


In [16]:
queries = read_qsym_log("./sample2_export.log")
# [q.to_json_serializable() for q in queries]
len(queries)

103

In [17]:
pool = SolverPool()
pool.selection_strategy = NegationSolverSelectionStrategy()

In [18]:
logging.getLogger().setLevel(level=logging.INFO)

for q in queries:
    pool.solve(q)

INFO:root:No solver available, Creating a new one
INFO:root:No solver available, Creating a new one
INFO:root:No solver available, Creating a new one
INFO:root:No solver available, Creating a new one


In [21]:
print("Solver tree count:", len(pool._solver_trees))
print("Most recently used solver:", list(pool._solvers.items.items())[-1][1]._solver)
print("Statistics:", [solver.statistics for (id, solver) in pool._solvers.items.items()])
s = list(pool._solvers.items.items())[-1][1]._solver
s.to_smt2()

Solver tree count: 1
Most recently used solver: [7 <= 2*Concat(3, 2, 1, 0)]
Statistics: [{'solve_count': 44, 'push_count': 0, 'pop_count': 0}, {'solve_count': 1, 'push_count': 1, 'pop_count': 1}, {'solve_count': 1, 'push_count': 1, 'pop_count': 1}, {'solve_count': 57, 'push_count': 42, 'pop_count': 42}]


'; benchmark generated from python API\n(set-info :status unknown)\n(declare-fun |0| () (_ BitVec 8))\n(declare-fun |1| () (_ BitVec 8))\n(declare-fun |2| () (_ BitVec 8))\n(declare-fun |3| () (_ BitVec 8))\n(assert\n (let ((?x11 (concat |3| |2| |1| |0|)))\n (let ((?x12 (bvmul (_ bv2 32) ?x11)))\n (bvsle (_ bv7 32) ?x12))))\n(check-sat)\n'